# TC Profiling: Service Performance Model



In [1]:
# global settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.interpolate import griddata
import numpy as np
import seaborn
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import networkx as nx
import itertools as it

In [2]:
def cartesian_product(p_dict):
    """
    Compute Cartesian product on parameter dict:
    In:
        {"number": [1,2,3], "color": ["orange","blue"] }
    Out:
        [ {"number": 1, "color": "orange"},
          {"number": 1, "color": "blue"},
          {"number": 2, "color": "orange"},
          {"number": 2, "color": "blue"},
          {"number": 3, "color": "orange"},
          {"number": 3, "color": "blue"}
        ]
    """
    p_names = sorted(p_dict)
    return [dict(zip(p_names, prod)) for prod in it.product(
        *(p_dict[n] for n in p_names))]

## Assumptions

## Model Design

### VNFs


### Service

In [3]:
class VnfPerformanceModel(object):
    
    def __init__(self, vnf_id, name, parameter, func):
        self.vnf_id = vnf_id  # identification
        self.name = name  # just for humans
        # list of parameter tuples (name, lst_of_values)
        self.parameter = parameter
        # function to evaluate the VNF's performance
        # (p1, ... pn) -> performance
        self.func = func
        print("Generated VNF {} with vnf_id={}".format(name, vnf_id))
        
    def evaluate(self, c):
        #print("eval VNF={} cf={}".format(self.name, c))
        return self.func(c)

class SfcPerformanceModel(object):

    @classmethod
    def generate(cls, conf):
        """
        Generate list of model objects. One for each conf. to be tested.
        """
        # TODO change to generate real multiple models based on cnfigs
        parameter, vnf_lst = cls.generate_vnfs(conf)
        sfc_graph = cls.generate_sfc_graph(conf, vnf_lst)
        print("Generated SFC graph with nodes={} and edges={}".format(
              sfc_graph.nodes(), sfc_graph.edges()))       
        pm_obj = cls(parameter=parameter, vnfs=vnf_lst, sfc_graph=sfc_graph)
        return [pm_obj]
    
    @classmethod
    def generate_vnfs(cls, conf):
        print("VNF generation not implemented in base class.")
        
    @classmethod
    def generate_sfc_graph(cls, conf):
        print("Service graph generation not implemented in base class.")
    
    def __init__(self, **kwargs):
        self.parameter = kwargs.get("parameter", {})
        self.vnfs = kwargs.get("vnfs", [])
        self.sfc_graph = kwargs.get("sfc_graph")
        print("Initialized performance model: '{}' with {} VNFs ...".format(
            self, len(self.vnfs)))
        print("\t ... the SFC graph has {} nodes and {} edges ...".format(
            len(self.sfc_graph.nodes()),  len(self.sfc_graph.edges())))
        print("\t ... each VNF has {} possible configurations ...".format(
            len(self.get_conf_space_vnf())))
        print("\t ... the SFC has {} possible configurations.".format(
            len(self.get_conf_space())))

    def __repr__(self):
        return "{}".format(
            self.name)

    @property
    def name(self):
        return self.__class__.__name__

    @property
    def short_name(self):
        return re.sub('[^A-Z]', '', self.name)

    def get_results(self):
        """
        Getter for global result collection.
        :return: dict for result row
        """
        r = {"pmodel": self.short_name}
        return r
    
    def get_conf_space_vnf(self):
        """
        Return the configuration space for a single VNF.
        :return: list of configuration dicts
        """
        return cartesian_product(self.parameter)

    def get_conf_space(self):
        """
        Return the COMPLETE configuration space for this model.
        :return: list of configuration tuples of one dict per VNF
        """
        # config space for one VNF
        cf = self.get_conf_space_vnf()
        # config space for n VNFs in the SFC
        cs = list(it.product(cf, repeat=len(self.vnfs)))
        #print(cs)
        return cs
           
    def evaluate(self, c):
        G = self.sfc_graph
        # 1. compute/assign capacity to each node
        print("eval cs={}".format(c))
        for n, nd in G.nodes(data=True):
            if nd.get("vnf") is not None:
                nd["capacity"] = nd.get("vnf").evaluate(c[n])
                print(n, nd)
        # 2. transform graph: reduce to classic max_flwo problem
        #    (each VNF node becomes vin - vout)
        G_new = nx.DiGraph()
        # for each node add in and out with capacity edge
        for (n, nd) in G.nodes(data="capacity"):
            G_new.add_node("{}_in".format(n), nd)
            G_new.add_node("{}_out".format(n), nd)
            G_new.add_edge("{}_in".format(n), "{}_out".format(n), capacity=nd.get("capacity", float("inf")))
        # replicate each edge from old graph in new graph
        for (u, v) in G.edges():
            G_new.add_edge("{}_out".format(u), "{}_in".format(v))
        
        print("Nodes:", G_new.nodes())
        for e in G_new.edges(data=True):
            print("Edge:", e)
            
        mf = nx.maximum_flow(G_new, "s_in", "t_out")
        print("")
        print("MaxFlow: {}".format(mf))
        print("")
        return None

    
class TestModel(SfcPerformanceModel):

    @classmethod
    def generate_vnfs(cls, conf):
        """
        A simplistic model to test implementation correctness.
        TODO shall be moved to a unittest
        """
        ## define parameters
        p = {"p1": [0, 1], "p2": [0, 1]}
        ## create vnfs
        vnf0 = VnfPerformanceModel(0, "vnf_0", p, lambda c: 5)
        vnf1 = VnfPerformanceModel(0, "vnf_1", p, lambda c: 3)
        vnf2 = VnfPerformanceModel(0, "vnf_2", p, lambda c: 4)
        vnf3 = VnfPerformanceModel(0, "vnf_3", p, lambda c: 9)
        # return parameters, list of vnfs
        return p, [vnf0, vnf1, vnf2, vnf3]
    
    @classmethod
    def generate_sfc_graph(cls, conf, vnfs):
        G = nx.DiGraph()
        # add nodes and assign VNF objects
        G.add_node(0, vnf=vnfs[0])
        G.add_node(1, vnf=vnfs[1])
        G.add_node(2, vnf=vnfs[2])
        G.add_node(3, vnf=vnfs[3])
        G.add_node("s", vnf=None)
        G.add_node("t", vnf=None)
        # two path diamond: s-0-(1/2)-3-t
        G.add_edges_from([("s", 0), (0, 1), (0, 2), (1, 3), (2, 3), (3, "t")])
        return G
    
######################################################## Test code for Jupyter ########################


#em = ExampleModel.generate(None)[0]  # use first generated model only
#cs = em.get_conf_space()

#print("\n Example model evaluation:")
#for i in range(0, len(cs), int(len(cs) / 12)):
#    print("{} -> {}".format(cs[i], em.evaluate(cs[i])))

print("")
print("-" * 70)
print("")    
    
tm = TestModel.generate(None)[0]  # use first generated model only
cs = tm.get_conf_space()

print("\n Test model evaluation:")
for i in range(0, len(cs), int(len(cs) / 1)):
    print("{} -> {}".format(cs[i], tm.evaluate(cs[i])))


----------------------------------------------------------------------

Generated VNF vnf_0 with vnf_id=0
Generated VNF vnf_1 with vnf_id=0
Generated VNF vnf_2 with vnf_id=0
Generated VNF vnf_3 with vnf_id=0
Generated SFC graph with nodes=[0, 1, 2, 3, 's', 't'] and edges=[(0, 1), (0, 2), (1, 3), (2, 3), (3, 't'), ('s', 0)]
Initialized performance model: 'TestModel' with 4 VNFs ...
	 ... the SFC graph has 6 nodes and 6 edges ...
	 ... each VNF has 4 possible configurations ...
	 ... the SFC has 256 possible configurations.

 Test model evaluation:
eval cs=({'p1': 0, 'p2': 0}, {'p1': 0, 'p2': 0}, {'p1': 0, 'p2': 0}, {'p1': 0, 'p2': 0})
0 {'vnf': <__main__.VnfPerformanceModel object at 0x7f5cf1882240>, 'capacity': 5}
1 {'vnf': <__main__.VnfPerformanceModel object at 0x7f5cf18822e8>, 'capacity': 3}
2 {'vnf': <__main__.VnfPerformanceModel object at 0x7f5cf1882390>, 'capacity': 4}
3 {'vnf': <__main__.VnfPerformanceModel object at 0x7f5cfb6012b0>, 'capacity': 9}
Nodes: ['0_in', '0_out', '1_i

# Backup

In [4]:
class ExampleModel(SfcPerformanceModel):

    @classmethod
    def generate_vnfs(cls, conf):
        """
        Example for defining VNFs of a SFC performance model.
        """
        ## define parameters
        # dict of lists defining possible configuration parameters
        p = {"cores": list(range(1, 17)),  # no
             "mem": [2**n for n in range(6, 16)],  # mb
             "blkio": [100, 200, 300, 400, 500],  # mb/s
             "sriov": [0, 1]}  # yes/no
        ## create vnfs
        # function: config_list -> performance
        # REQUIREMENT: vnf_ids of objects == idx in list
        vnf0 = VnfPerformanceModel(0, "vnf_0", p,
            lambda c: (c[0] * 8.0 + c[1] * 1.5 + c[2] * 0.0) * (1.0 + c[4]))    
        vnf1 = VnfPerformanceModel(1, "vnf_1", p,
            lambda c: (c[0] * 4.0 + c[1] * .5 + c[2] * 5.0)) 
        vnf2 = VnfPerformanceModel(2, "vnf_2", p,
            lambda c: (c[0] * 2.0 + c[1] * 2.5 + c[2] * 0.0) * (.5 + c[4]))
        vnf3 = VnfPerformanceModel(3, "vnf_3", p,
            lambda c: (c[0] * 1.0 + c[1] * 0 + c[2] * 2.0))
        # return parameters, list of vnfs
        return p, [vnf0, vnf1, vnf2, vnf3]
    
    @classmethod
    def generate_sfc_graph(cls, conf, vnfs):
        G = nx.Graph()
        # add nodes and assign VNF objects
        G.add_node(0, vnf=vnfs[0])
        G.add_node(1, vnf=vnfs[1])
        G.add_node(2, vnf=vnfs[2])
        G.add_node(3, vnf=vnfs[3])
        G.add_node(4, vnf=vnfs[1])
        G.add_node("s", vnf=None)
        G.add_node("t", vnf=None)
        # simple linear: 0 -> 1 -> 2 -> 3 -> 1
        G.add_edges_from([("s", 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, "t")])
        return G